In [27]:
import pandas as pd
import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, rdMolDescriptors
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import StratifiedKFold
import gensim
from gensim import models
import time
import pickle
import json
import urllib
import requests
import os
import gzip

import zipfile


global_random_state = 42
k_fold_splits = 2

In [29]:
gene_symbol = "PPARG"
# Use an API call to find all related bioassays. For now, use the assays related to PPAR gamma
assays_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/target/genesymbol/{0}/aids/TXT".format(gene_symbol)
r = requests.get(assays_url)
relevant_assays = [int(x) for x in r.text.split('\n') if len(x) > 0]
print(len(relevant_assays))

assay_dir = "/media/data/pubchem/Data"

# Each assay n is in a file starting from 0001 to 1000, etc

assay_paths = list()

dfs = list()

for assay_num in relevant_assays:
    print(assay_num)
    # Round assay into down to nearest thousand
    assay_num_rounded_lower = assay_num - (assay_num % 1000) + 1
    assay_num_rounded_upper = assay_num_rounded_lower + 999
    expected_folder_name = "{0:0>7}_{1:0>7}".format(assay_num_rounded_lower,assay_num_rounded_upper)
    expected_name = "{0}.zip".format(expected_folder_name)
    expected_path = os.path.join(assay_dir,expected_name)
    assay_paths.append(expected_path)
    archive = zipfile.ZipFile(expected_path, 'r')
    
    with archive.open(expected_folder_name + '/' + str(assay_num) + ".csv.gz") as f:
        with gzip.open(f) as g:
            
            df = pd.read_csv(g)
            dfs.append(df)
            print("Added assay: "+str(assay_num))


#aid = 1032
# Use an API call to find all related bioassays. For now, use the assays related to PPAR gamma
#assay_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/{0}/description/JSON".format(aid)
#json.load(urllib.urlopen(assay_url))

912
631
Added assay: 631
731


/home/joe/anaconda3/envs/cs510-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Added assay: 731
1032
Added assay: 1032
1048
Added assay: 1048
1049
Added assay: 1049
1051
Added assay: 1051
1297
Added assay: 1297
1300
Added assay: 1300
1301
Added assay: 1301
1319
Added assay: 1319
1323
Added assay: 1323
1331
Added assay: 1331
1679
Added assay: 1679
1681
Added assay: 1681
1684
Added assay: 1684
1808
Added assay: 1808
1810
Added assay: 1810
1812
Added assay: 1812
2277
Added assay: 2277
2449
Added assay: 2449
2759
Added assay: 2759
46660
Added assay: 46660
46661
Added assay: 46661
91246
Added assay: 91246
93033
Added assay: 93033
141913
Added assay: 141913
142314
Added assay: 142314
155978
Added assay: 155978
155979
Added assay: 155979
156360
Added assay: 156360
156361
Added assay: 156361
156363
Added assay: 156363
156364
Added assay: 156364
156365
Added assay: 156365
156366
Added assay: 156366
156367
Added assay: 156367
156370
Added assay: 156370
156371
Added assay: 156371
156372
Added assay: 156372
156373
Added assay: 156373
156376
Added assay: 156376
156377
Added a

Added assay: 361858
361859
Added assay: 361859
361862
Added assay: 361862
361863
Added assay: 361863
361864
Added assay: 361864
361865
Added assay: 361865
361866
Added assay: 361866
361867
Added assay: 361867
361869
Added assay: 361869
364306
Added assay: 364306
364307
Added assay: 364307
364308
Added assay: 364308
364309
Added assay: 364309
364310
Added assay: 364310
364311
Added assay: 364311
364312
Added assay: 364312
364314
Added assay: 364314
364412
Added assay: 364412
364413
Added assay: 364413
364414
Added assay: 364414
371250
Added assay: 371250
371254
Added assay: 371254
372041
Added assay: 372041
372044
Added assay: 372044
372045
Added assay: 372045
372047
Added assay: 372047
372057
Added assay: 372057
375473
Added assay: 375473
375478
Added assay: 375478
375479
Added assay: 375479
381056
Added assay: 381056
381057
Added assay: 381057
381065
Added assay: 381065
382295
Added assay: 382295
382296
Added assay: 382296
382299
Added assay: 382299
382314
Added assay: 382314
387492
A

Added assay: 699015
699016
Added assay: 699016
702659
Added assay: 702659
702665
Added assay: 702665
703030
Added assay: 703030
703034
Added assay: 703034
708112
Added assay: 708112
708113
Added assay: 708113
708114
Added assay: 708114
708115
Added assay: 708115
708116
Added assay: 708116
708117
Added assay: 708117
708118
Added assay: 708118
708119
Added assay: 708119
708121
Added assay: 708121
708124
Added assay: 708124
708125
Added assay: 708125
709646
Added assay: 709646
709648
Added assay: 709648
712380
Added assay: 712380
712383
Added assay: 712383
712384
Added assay: 712384
712387
Added assay: 712387
712390
Added assay: 712390
712391
Added assay: 712391
712392
Added assay: 712392
715913
Added assay: 715913
715946
Added assay: 715946
717961
Added assay: 717961
718080
Added assay: 718080
718081
Added assay: 718081
718849
Added assay: 718849
718850
Added assay: 718850
721111
Added assay: 721111
722375
Added assay: 722375
722377
Added assay: 722377
722385
Added assay: 722385
722386
A

/home/joe/anaconda3/envs/cs510-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,8,9,10,11,13,14,15,16,17,18,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,48,49,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,84,85,86,87,88,89,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,2

Added assay: 743094
743140
Added assay: 743140
743191


/home/joe/anaconda3/envs/cs510-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0,8,9,10,11,13,14,15,16,17,18,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,48,49,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,25

Added assay: 743191
743199
Added assay: 743199
744058
Added assay: 744058
744059
Added assay: 744059
744060
Added assay: 744060
744061
Added assay: 744061
744063
Added assay: 744063
744064
Added assay: 744064
744065
Added assay: 744065
744324
Added assay: 744324
746498
Added assay: 746498
746612
Added assay: 746612
746614
Added assay: 746614
746618
Added assay: 746618
751861
Added assay: 751861
752224
Added assay: 752224
752293
Added assay: 752293
757820
Added assay: 757820
757821
Added assay: 757821
757822
Added assay: 757822
765265
Added assay: 765265
765266
Added assay: 765266
768850
Added assay: 768850
768851
Added assay: 768851
772910
Added assay: 772910
772911
Added assay: 772911
775854
Added assay: 775854
775855
Added assay: 775855
1060242
Added assay: 1060242
1062216
Added assay: 1062216
1063306
Added assay: 1063306
1063307
Added assay: 1063307
1063308
Added assay: 1063308
1063309
Added assay: 1063309
1063310
Added assay: 1063310
1063311
Added assay: 1063311
1063312
Added assay

In [ ]:
fingerprints_all = list()
activities_all = list()

In [ ]:
df = pd.read_csv("AID_1032_datatable_all.csv")
df.head()

In [3]:
# Merge in the SMILES info for all compound IDs
cs = pd.read_csv("1032_CID_SMILES_Mapping.csv",sep='\t',header=0)
df = df.merge(cs,on="PUBCHEM_CID")
df.head()

,PUBCHEM_RESULT_TAG,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_URL,PUBCHEM_ASSAYDATA_COMMENT,Activation,SMILES
0,1,842121.0,6.60301e+06,Inactive,6.0,NaN,NaN,6.82,CCOCCCNCC(=O)NC1=CC=C(C=C1)OC(F)(F)F.Cl
1,2,842122.0,6.60257e+06,Inactive,2.0,NaN,NaN,2.45,COCCN1C(=NN=N1)CN2CCC(CC2)CC3=CC=CC=C3.Cl
2,3,842123.0,6.60262e+06,Inactive,1.0,NaN,NaN,1.41,COCCN1C(=NN=N1)CN2CCC(CC2)(C3=CC(=CC=C3)C(F)(F...
3,4,842124.0,644371,Inactive,0.0,NaN,NaN,-5.09,C1CCCN(CC1)CC(=O)NCCC2=CC=C(C=C2)F.C(=O)(C(=O)O)O
4,5,842125.0,6.60313e+06,Inactive,0.0,NaN,NaN,-2.24,COC1=CC=C(C=C1)C(=O)C(C2=CC=CC=C2)N3CCOCC3.Cl


In [4]:
# Binarize the inactive/active column

df["IS_ACTIVE"] = df["PUBCHEM_ACTIVITY_OUTCOME"] == "Active"
df["IS_ACTIVE"].astype(bool)

df_active = df[df["IS_ACTIVE"] == True]
df_inactive = df[df["IS_ACTIVE"] == False]

df["IS_ACTIVE"].head()

0    False
1    False
2    False
3    False
4    False
Name: IS_ACTIVE, dtype: bool

In [5]:
# Plot histogram of active
%matplotlib inline

print("Active are: {}, Inactive are: {}".format(df_active.count()["PUBCHEM_CID"], df_inactive.count()["PUBCHEM_CID"]))

Active are: 670, Inactive are: 195584


In [19]:
# Now let's calculate fixed-length features which we need to train an ML algorithm

cids = list()
fingerprints = list()
activities = list()
mols = list()

num_parsed = 0
num_active = 0
num_inactive = 0

for index, row in df.iterrows() :
    cid = row["PUBCHEM_CID"]
    smiles_string = row["SMILES"]
    mol = Chem.MolFromSmiles(smiles_string)
    is_active = row["PUBCHEM_ACTIVITY_OUTCOME"] == "Active"
    
    class_size = 670
    
    if num_active == class_size and num_inactive == class_size :
        print("We have enough data")
        break
    
    if mol is None:
        print("Molecule failed featurization")
        print(index)
    else: 
        fingerprint = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol,2,nBits=2048,useChirality=False,
                                                                     useBondTypes=False,useFeatures=False)
        # From RDKit documentation
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fingerprint, arr)
        fingerprint = arr
        
        
        
        if(is_active and num_active < class_size) :
            cids.append(cid)
            num_active = num_active + 1
            fingerprints.append(fingerprint)
            activities.append(is_active)
            mols.append(mol)
        elif (not is_active and num_inactive < class_size) :
            cids.append(cid)
            num_inactive = num_inactive + 1
            fingerprints.append(fingerprint)
            activities.append(is_active)
            mols.append(mol)
            
        num_parsed = num_parsed + 1
    
    if index % 10000 == 0:
        print("Processed index: {0}".format(index))
        print("Active are: {}, Inactive are: {}".format(num_active,num_inactive))

X = np.array(fingerprints)
y = np.array(activities)
        
fingerprints = np.array(fingerprints)
activities = np.array(activities,dtype=int)

rus = RandomUnderSampler(random_state=global_random_state)
X, y = rus.fit_sample(fingerprints, activities)

# Pickle the data to save time in the future
with open('data.classification.undersampled.1032.pickle', 'wb') as f:
    pickle.dump((X,y), f, pickle.HIGHEST_PROTOCOL)

Processed index: 0
Active are: 0, Inactive are: 1
Processed index: 10000
Active are: 16, Inactive are: 670
Processed index: 20000
Active are: 81, Inactive are: 670
Processed index: 30000
Active are: 105, Inactive are: 670
Processed index: 40000
Active are: 173, Inactive are: 670
Processed index: 50000
Active are: 204, Inactive are: 670
Processed index: 60000
Active are: 234, Inactive are: 670
Processed index: 70000
Active are: 269, Inactive are: 670
Processed index: 80000
Active are: 316, Inactive are: 670
Processed index: 90000
Active are: 368, Inactive are: 670
Processed index: 100000
Active are: 399, Inactive are: 670
Processed index: 110000
Active are: 431, Inactive are: 670
Processed index: 120000
Active are: 463, Inactive are: 670
Processed index: 130000
Active are: 497, Inactive are: 670
Processed index: 140000
Active are: 521, Inactive are: 670
Processed index: 150000
Active are: 555, Inactive are: 670
Processed index: 160000
Active are: 577, Inactive are: 670
Processed index: 

In [21]:
with open('data.classification.undersampled.1032.pickle','rb') as f:
    (X,y) = pickle.load(f)

In [24]:
# Note - minor code used from https://github.com/LRParser/pubchem-sklearn/blob/master/pubchem_bioassay_sklearn.ipynb
skf = StratifiedKFold(n_splits=k_fold_splits,shuffle=True,random_state=global_random_state)

roc_auc_avg = 0
avg_predict_time = 0
for train_index, test_index in skf.split(X,y) :

    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    
    print("Number of training samples is: {}".format(len(X_train)))
    print("Number of test samples is: {}".format(len(X_test)))

    classifier = DummyClassifier(random_state=global_random_state)
    classifier.fit(X_train,y_train)
    start = time.time()
    y_pred = classifier.predict(X_test)
    elapsed = time.time() - start
    avg_predict_time = avg_predict_time + elapsed
    auc = roc_auc_score(y_test, y_pred, average='macro', sample_weight=None)
    roc_auc_avg = roc_auc_avg + auc
    
avg_predict_time = avg_predict_time / k_fold_splits
roc_auc_avg = roc_auc_avg / k_fold_splits
print("Average roc_auc score of {} folds is: {}".format(k_fold_splits, roc_auc_avg))
print("Average elapsed prediction time over {} folds in s is: {}".format(k_fold_splits, avg_predict_time))

Number of training samples is: 670
Number of test samples is: 670
Number of training samples is: 670
Number of test samples is: 670
Average roc_auc score of 2 folds is: 0.5149253731343284
Average elapsed prediction time over 2 folds in s is: 0.0019965171813964844


In [25]:
roc_auc_avg = 0
avg_predict_time = 0
for train_index, test_index in skf.split(X,y) :

    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    
    print("Number of training samples is: {}".format(len(X_train)))
    print("Number of test samples is: {}".format(len(X_test)))

    classifier = RandomForestClassifier(random_state=global_random_state)
    classifier.fit(X_train,y_train)
    start = time.time()
    y_pred = classifier.predict(X_test)
    elapsed = time.time() - start
    avg_predict_time = avg_predict_time + elapsed
    auc = roc_auc_score(y_test, y_pred, average='macro', sample_weight=None)
    roc_auc_avg = roc_auc_avg + auc
    
avg_predict_time = avg_predict_time / k_fold_splits
roc_auc_avg = roc_auc_avg / k_fold_splits
print("Average roc_auc score of {} folds is: {}".format(k_fold_splits, roc_auc_avg))
print("Average elapsed prediction time over {} folds in s is: {}".format(k_fold_splits, avg_predict_time))

Number of training samples is: 670
Number of test samples is: 670
Number of training samples is: 670
Number of test samples is: 670
Average roc_auc score of 2 folds is: 0.8223880597014925
Average elapsed prediction time over 2 folds in s is: 0.003376007080078125


In [29]:
# Let's check out training on embeddings

model = models.KeyedVectors.load_word2vec_format("vec.txt")
embeddings = list()

for mol in mols:
    try :
        info = {}
        rdMolDescriptors.GetMorganFingerprint(mol,0,bitInfo=info)
        keys = info.keys()
        keys_list = list(keys)
        totalvec = np.zeros(200)
        for k in keys_list:
            wordvec = model.wv[str(k)]
            totalvec = np.add(totalvec,wordvec)
        #print(totalvec)
        embeddings.append(totalvec)
    except Exception as e:
        print(e)
        pass
    
print(len(embeddings))
X = np.array(embeddings)



1340


In [30]:
# Now let's look at performance with embeddings, 70 MB

roc_auc_avg = 0
avg_predict_time = 0
for train_index, test_index in skf.split(X,y) :

    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    
    print("Number of training samples is: {}".format(len(X_train)))
    print("Number of test samples is: {}".format(len(X_test)))

    classifier = RandomForestClassifier(random_state=global_random_state)
    classifier.fit(X_train,y_train)
    start = time.time()
    y_pred = classifier.predict(X_test)
    elapsed = time.time() - start
    avg_predict_time = avg_predict_time + elapsed
    auc = roc_auc_score(y_test, y_pred, average='macro', sample_weight=None)
    roc_auc_avg = roc_auc_avg + auc
    
avg_predict_time = avg_predict_time / k_fold_splits
roc_auc_avg = roc_auc_avg / k_fold_splits
print("Average roc_auc score of {} folds is: {}".format(k_fold_splits, roc_auc_avg))
print("Average elapsed prediction time over {} folds in s is: {}".format(k_fold_splits, avg_predict_time))

Number of training samples is: 670
Number of test samples is: 670
Number of training samples is: 670
Number of test samples is: 670
Average roc_auc score of 2 folds is: 0.7291044776119403
Average elapsed prediction time over 2 folds in s is: 0.0015490055084228516


In [31]:
# Now let's look at performance with embeddings, 90 MB

model = models.KeyedVectors.load_word2vec_format("vec.txt")
embeddings = list()

for mol in mols:
    try :
        info = {}
        rdMolDescriptors.GetMorganFingerprint(mol,1,bitInfo=info)
        keys = info.keys()
        keys_list = list(keys)
        totalvec = np.zeros(200)
        for k in keys_list:
            wordvec = model.wv[str(k)]
            totalvec = np.add(totalvec,wordvec)
        #print(totalvec)
        embeddings.append(totalvec)
    except Exception as e:
        print(e)
        pass
    
print(len(embeddings))
X = np.array(embeddings)

roc_auc_avg = 0
avg_predict_time = 0
for train_index, test_index in skf.split(X,y) :

    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    
    print("Number of training samples is: {}".format(len(X_train)))
    print("Number of test samples is: {}".format(len(X_test)))

    classifier = RandomForestClassifier(random_state=global_random_state)
    classifier.fit(X_train,y_train)
    start = time.time()
    y_pred = classifier.predict(X_test)
    elapsed = time.time() - start
    avg_predict_time = avg_predict_time + elapsed
    auc = roc_auc_score(y_test, y_pred, average='macro', sample_weight=None)
    roc_auc_avg = roc_auc_avg + auc
    
avg_predict_time = avg_predict_time / k_fold_splits
roc_auc_avg = roc_auc_avg / k_fold_splits
print("Average roc_auc score of {} folds is: {}".format(k_fold_splits, roc_auc_avg))
print("Average elapsed prediction time over {} folds in s is: {}".format(k_fold_splits, avg_predict_time))

1340
Number of training samples is: 670
Number of test samples is: 670
Number of training samples is: 670
Number of test samples is: 670
Average roc_auc score of 2 folds is: 0.7552238805970151
Average elapsed prediction time over 2 folds in s is: 0.00133514404296875


In [32]:
# Now let's look at performance with full embeddings

model = models.KeyedVectors.load_word2vec_format("vec.txt")
embeddings = list()

for mol in mols:
    try :
        info = {}
        rdMolDescriptors.GetMorganFingerprint(mol,1,bitInfo=info)
        keys = info.keys()
        keys_list = list(keys)
        totalvec = np.zeros(200)
        for k in keys_list:
            wordvec = model.wv[str(k)]
            totalvec = np.add(totalvec,wordvec)
        #print(totalvec)
        embeddings.append(totalvec)
    except Exception as e:
        print(e)
        pass
    
print(len(embeddings))
X = np.array(embeddings)

roc_auc_avg = 0
avg_predict_time = 0
for train_index, test_index in skf.split(X,y) :

    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    
    print("Number of training samples is: {}".format(len(X_train)))
    print("Number of test samples is: {}".format(len(X_test)))

    classifier = RandomForestClassifier(random_state=global_random_state)
    classifier.fit(X_train,y_train)
    start = time.time()
    y_pred = classifier.predict(X_test)
    elapsed = time.time() - start
    avg_predict_time = avg_predict_time + elapsed
    auc = roc_auc_score(y_test, y_pred, average='macro', sample_weight=None)
    roc_auc_avg = roc_auc_avg + auc
    
avg_predict_time = avg_predict_time / k_fold_splits
roc_auc_avg = roc_auc_avg / k_fold_splits
print("Average roc_auc score of {} folds is: {}".format(k_fold_splits, roc_auc_avg))
print("Average elapsed prediction time over {} folds in s is: {}".format(k_fold_splits, avg_predict_time))

1340
Number of training samples is: 670
Number of test samples is: 670
Number of training samples is: 670
Number of test samples is: 670
Average roc_auc score of 2 folds is: 0.7417910447761195
Average elapsed prediction time over 2 folds in s is: 0.0013414621353149414
